# Lab-11: Ensemble Learning

```
- Machine Learning, Innopolis University (Fall semester 2021)
- Professor: Adil Khan
- Teaching Assistant: Gcinizwe Dlamini
```
<hr>


```
In this lab, you will practice Ensemble learning

Lab Plan : Ensemble learning
1. Bagging 
2. Random Forests
3. AdaBoost
```

<hr>

## 1. Ensemble learning


Why ensemble learning? How does it help?


We will explore ensemble learning on the example of decision trees - we will see how ensembles can improve classification accuracy.

Let's start from uploading MNIST dataset.

In [ ]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

digits = load_digits()
X = digits.data
y = digits.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)


plt.figure(1, figsize=(3, 3))
plt.imshow(X[0].reshape((8,8)), cmap="gray")
plt.xticks([])
plt.yticks([])
plt.title(f"label is {y[0]}")
plt.show()

## Recap : Single decision tree

First, we train a single decision tree.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)
pred = tree.predict(X_test)
tree_score = accuracy_score(y_test, pred)
print("Single tree accuracy:", tree_score)

Note the accuracy - it is around **0.85**.

## 1. Bagging


What is decreased by bagging? Variance or bias? How? 

Now let's improve it a bit by the means of bagging. We train a hundred of independent classifiers and make a prediction by majority voting.

In [ ]:
import numpy as np
from scipy.stats import mode

n_trees = 100

classifiers = []
for i in range(n_trees):
    # TODO: train a new classifier and append it to the list
    pass


# here we will store predictions for all samples and all base classifiers
base_pred = np.zeros((X_test.shape[0], n_trees), dtype="int")
for i in range(n_trees):
    # TODO: obtain the predictions from each tree
    base_pred[:,i] = None

print(base_pred)

# aggregate predictions by majority voting
pred = mode(base_pred, axis=1)[0].ravel()
acc = accuracy_score(y_test, pred)
print("Bagging accuracy:", acc)

Now the accuracy grew up to **0.88**. You can see that our classifiers return very similar results. By the way, why the base classifiers are not identical at all?


## 2. Random forest

Compared to simple bagging we've just implemented, random forest can show better results because base classifiers are much less correlated.

At first, let's implement bootstrap sampling.

### Task : implement bootstrap sampling (use numpy)

In [ ]:
def bootstrap(X, y):
    # TODO: generate bootstrap indices and return data according to them
    X_bs = None
    y_bs = None
    return X_bs, y_bs


# this is a test, will work if you are using np.random.randint() for indices generation
np.random.seed(0)
a = np.array(range(12)).reshape(4,3)
b = np.array(range(4))
bootstrap(a, b)

You should get

(array([[ 0,  1,  2], <br>
&emsp;&emsp;&emsp;[ 9, 10, 11], <br>
&emsp;&emsp;&emsp;[ 3,  4,  5], <br>
&emsp;&emsp;&emsp;[ 0,  1,  2]]), <br>
array([0, 3, 1, 0]))
       
## 2.1 Random forest from scratch

Now let's build a set of decision trees, each of them is trained on a bootstrap sampling from X and $\sqrt d$ features.

### Task : Build Random forest from scratch

In [ ]:
classifiers = []
for i in range(n_trees):
    # TODO: train a new tree on sqrt(n_features) and bootstrapped data, append it to the list
    pass

base_pred = np.zeros((n_trees, X_test.shape[0]), dtype="int")
for i in range(n_trees):
    base_pred[i,:] = classifiers[i].predict(X_test)

pred = mode(base_pred, axis=0)[0].ravel()
acc = accuracy_score(y_test, pred)
print("Random forest accuracy:", acc)

And now we got **0.97** accuracy, which is a significant improvement! Now you can see why it is so important to have diverse classifiers.

## 3. Boosting

How does boosting work? 

For simplicity let's make a binary classification problem out of the original problem.

In [ ]:
y_train_b = (y_train == 2 ) * 2 - 1
y_test_b = (y_test == 2 ) * 2 - 1

Now let's train a boosting model.

We will have sample weights and tree weights. Initially all sample weights are equal. After that we will increase weight for complicated samples.

Tree weight $w$ is computed using weighted error or $1 - accuracy$

$w_t = \frac12 log(\frac{1-weighted\_error_t}{weighted\_error_t})$ for each base classifier.

For correct samples weights will be decreased $e^w$ times, and for incorrect classified samples increased  $e^w$ times. After this changes we normalize weights.

## 3.1 Boosting from Scratch 

Tasks: 
1. initialize sample weights
1. caclulate tree weight
1. update sample weights
1. normalize the weights

In [ ]:
n_trees = 10
tree_weights = np.zeros(n_trees)
classifiers = []
train_samples = X_train.shape[0]
# TODO : initialize sample weights
sample_weights = None
for i in range(n_trees):
    clf = DecisionTreeClassifier(min_samples_leaf=3)
    clf.fit(X_train, y_train_b, sample_weight=sample_weights)
    pred = clf.predict(X_train)
    acc = accuracy_score(y_train_b, pred, sample_weight=sample_weights)
    # TODO: caclulate tree weight
    w = None
    tree_weights[i] = None
    classifiers.append(clf)
    # update sample weights
    for j in range(train_samples):
        if pred[j] != y[j]:
            # in case of a misclassification
            pass
        else:
            # in case when the prediction is correct
            pass
    # normalize the weights
    sample_weights = None

Use trees voting to calculate final predictions. Since we have a binary classification, the prediction will be calculated as follows:

$\hat{y} = sign(\sum_{t=1}^{T}(w_t f_t(x)))$

In [ ]:
n_test = X_test.shape[0]

pred = np.zeros(n_test)
# TODO: caclulate predictions

acc = accuracy_score(y_test_b, pred)
print("Boosting accuracy:", acc)

The resulting accuracy is **0.97**

## Optional : Compare your implementation with sklearn 